# Baseline Metrics of Search User Engagement Metrics


[T289601](https://phabricator.wikimedia.org/T289601)

In this notebook, we establish baselines for search user engagement metrics for the Search Platform team.

The metrics are (which we are tracking in dashbaord):
- Number of search sessions (including full-text sessions, click sessions and checkin sessions).
- Number of sessions with dwell time >= 10s.
- Proportion of sessions with dwell time >= 10s.

The projects we are tracking include:
- Overall.
- Major projects (from top 10 wikis): commons, wikidata, enwiki, eswiki, dewiki, jawiki, ruwiki, itwiki, zhwiki, ptwiki.
- Projects of languages used in emerging market countries (Refer to [this sheet](https://docs.google.com/spreadsheets/d/14QzqCQeJbdfbXZOHDTYT5Ojd8pdisMpps37J7tvbNYo/edit?usp=sharing) for a detailed list of countries and languages).



## Notes

- Event logging in SearchSatisfaction is only done on the desktop platform. This means that for these metrics, platform will always be desktop. 

- For the baseline, we only look at user activities, which means the searches done by a bot or someone - who's likely a bot will not be included. If more than 50 searches are done in a session, it's likely a non-human agent and we'll label that automated. Otherwise, we'll label it user.

- The timeframe is from 09/01/2021 to 09/30/2021 as the activities picked in from summer/vacation time.

- We use median for daily metrics.



In [5]:
import datetime as dt

import pandas as pd
import numpy as np

from wmfdata import spark, mariadb, hive

import gspread
from google.oauth2.service_account import Credentials

In [6]:
# major projects that not in emerging coutries' language list
major_project_list = '"commonswiki","wikidatawiki","dewiki","itwiki","ruwiki"'

In [7]:
emerging_list = spark.run('''
SELECT project FROM cchen_search.emerging_market_list GROUP BY project
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [8]:
emerging_project_list = str(emerging_list.project.tolist())[1:-1]

## Number of search sessions per Day

In [11]:
search_session_query = '''

SELECT 
  project_family,
  wiki_db,
  f.language_code,
  PERCENTILE(f.num_fulltext_sessions,0.5) AS fulltext_sessions,
  PERCENTILE(f.num_click_sessions, 0.5) AS click_sessions,
  PERCENTILE(f.num_checkin_sessions,0.5) AS checkin_sessions
FROM search_dashboard_data.fulltext_funnel_counts f
WHERE log_date >= "2021-09-01"
  AND log_date < "2021-10-01"
  AND (f.wiki_db IN ({major}) OR f.wiki_db IN ({emerging}))
  AND agent_type = 'user'
GROUP BY f.project_family, f.wiki_db,f.language_code
'''

In [19]:
search_session_all_query = '''
WITH daily AS (
   SELECT 
     log_date, 
     SUM(num_fulltext_sessions) AS num_fulltext_sessions,
     SUM(num_click_sessions) AS num_click_sessions,
     SUM(num_checkin_sessions) AS num_checkin_sessions
   FROM search_dashboard_data.fulltext_funnel_counts
   WHERE log_date >= "2021-09-01"
     AND log_date < "2021-10-01"
     AND agent_type = 'user'
   GROUP BY log_date
)

SELECT 
  'Overall' AS project_family,
  'Overall' AS wiki_db,
  'Overall' AS language_code,
  PERCENTILE(num_fulltext_sessions, 0.5) AS fulltext_sessions,
  PERCENTILE(num_click_sessions, 0.5) AS click_sessions,
  PERCENTILE(num_checkin_sessions, 0.5) AS checkin_sessions
FROM daily
'''

In [15]:
search_session_daily = spark.run(search_session_query.format(major = major_project_list, emerging = emerging_project_list))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [20]:
search_session_overall_daily = spark.run(search_session_all_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


## Sessions with dwell time >= 10s per day

In [33]:
dwell_session_query = '''

WITH daily AS (
  SELECT 
    log_date,
    project_family,
    wiki_db,
    f.language_code,
    SUM(IF(max_checkin >= 10, num_sessions, 0)) AS num_dwell,
    ROUND(100 * SUM(IF(max_checkin >= 10, num_sessions, 0)) /
        SUM(num_sessions),2) AS percent_dwell
FROM search_dashboard_data.fulltext_checkin_counts f
WHERE log_date >= "2021-09-01"
  AND log_date < "2021-10-01"
  AND (f.wiki_db IN ({major}) OR f.wiki_db IN ({emerging}))
  AND agent_type = 'user'
GROUP BY log_date, f.project_family, f.wiki_db,f.language_code

)

SELECT 
  project_family,
  wiki_db,
  language_code,
  PERCENTILE(num_dwell, 0.5) AS num_dwell,
  PERCENTILE(percent_dwell, 0.5) AS percent_dwell
FROM daily
GROUP BY project_family, wiki_db,language_code
'''

In [35]:
dwell_session_all_query = '''

WITH daily AS (
  SELECT 
    log_date,
    SUM(IF(max_checkin >= 10, num_sessions, 0)) AS num_dwell,
    ROUND(100 * SUM(IF(max_checkin >= 10, num_sessions, 0)) /
        SUM(num_sessions),2) AS percent_dwell
FROM search_dashboard_data.fulltext_checkin_counts f
WHERE log_date >= "2021-09-01"
  AND log_date < "2021-10-01"
  AND agent_type = 'user'
GROUP BY log_date
)

SELECT 
  'Overall' AS project_family,
  'Overall' AS wiki_db,
  'Overall' AS language_code,
  PERCENTILE(num_dwell, 0.5) AS num_dwell,
  PERCENTILE(percent_dwell, 0.5) AS percent_dwell
FROM daily
'''

In [34]:
dwell_session_daily = spark.run(dwell_session_query.format(major = major_project_list, emerging = emerging_project_list))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [36]:
dwell_session_overall_daily = spark.run(dwell_session_all_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


## export to google sheet

In [38]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('big-mender-274521-324384a764e2.json', scopes=scope)
gc = gspread.authorize(credentials)

In [39]:
@property
def sheet1(self):
    """Shortcut property for getting the first worksheet."""
    return self.get_worksheet(0)

In [45]:
wks1 = gc.open("search_engagement_baselines").get_worksheet(0)
wks1.update([search_session_daily.columns.values.tolist()] + search_session_daily.values.tolist())

{'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
 'updatedRange': 'Sheet4!A1:F38',
 'updatedRows': 38,
 'updatedColumns': 6,
 'updatedCells': 228}

In [46]:
wks1.append_row(search_session_overall_daily.iloc[-1].tolist())

{'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
 'tableRange': 'Sheet4!A1:F38',
 'updates': {'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
  'updatedRange': 'Sheet4!A39:F39',
  'updatedRows': 1,
  'updatedColumns': 6,
  'updatedCells': 6}}

In [43]:
wks2 = gc.open("search_engagement_baselines").get_worksheet(1)
wks2.update([dwell_session_daily.columns.values.tolist()] + dwell_session_daily.values.tolist())

{'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
 'updatedRange': 'Sheet3!A1:E38',
 'updatedRows': 38,
 'updatedColumns': 5,
 'updatedCells': 190}

In [44]:
wks2.append_row(dwell_session_overall_daily.iloc[-1].tolist())

{'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
 'tableRange': 'Sheet3!A1:E38',
 'updates': {'spreadsheetId': '1FOI2nLcuaj4CPY-G0Uz460oM_2UIPTViYRAFWAXrQ6M',
  'updatedRange': 'Sheet3!A39:E39',
  'updatedRows': 1,
  'updatedColumns': 5,
  'updatedCells': 5}}